# CNN Encoder + LSTM Decoder
Custom 3-layer CNN encoder with LSTM decoder for handwritten math to LaTeX.

In [ ]:
!pip install -q datasets

In [ ]:
import numpy as np
import pickle
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from datasets import load_dataset
from pathlib import Path

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

c:\Users\dustp\anaconda3\envs\cs175_gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


## 1. Load and preprocess dataset

In [ ]:
ds = load_dataset("deepcopy/MathWriting-human")
print(f"Train: {len(ds['train'])}  Val: {len(ds['val'])}  Test: {len(ds['test'])}")

In [ ]:
# set to None to use ALL training data
NUM_TRAIN = 2000

ds_train = ds["train"] if NUM_TRAIN is None else ds["train"].select(range(NUM_TRAIN))
print(f"Using {len(ds_train)} training samples")

In [ ]:
IMG_SIZE = 128

images, latex_strings = [], []
for sample in ds_train:
    img = sample["image"].convert("L").resize((IMG_SIZE, IMG_SIZE))
    images.append(np.array(img) / 255.0)
    latex_strings.append(sample["latex"])

images = np.array(images)
print(f"Images: {images.shape}")

# char-level tokenizer
chars = sorted(set("".join(latex_strings)))
word_index = {ch: i + 1 for i, ch in enumerate(chars)}
idx_to_char = {v: k for k, v in word_index.items()}

sequences = [[word_index[ch] for ch in s] for s in latex_strings]
max_len = max(len(s) for s in sequences)
padded = np.array([s + [0] * (max_len - len(s)) for s in sequences])

VOCAB_SIZE = len(word_index) + 1
print(f"Vocab size: {VOCAB_SIZE}, Max seq len: {max_len}")

images_tensor = torch.tensor(images, dtype=torch.float32).unsqueeze(1)  # (N, 1, H, W)
tokens_tensor = torch.tensor(padded, dtype=torch.long)
print(f"Images tensor: {images_tensor.shape}")
print(f"Tokens tensor: {tokens_tensor.shape}")

dataset = TensorDataset(images_tensor, tokens_tensor)

## 2. Model definition

In [ ]:
class CNNEncoder(nn.Module):
    def __init__(self, encoded_dim=256):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, encoded_dim)

    def forward(self, x):
        features = self.cnn(x)
        features = self.adaptive_pool(features)
        features = features.view(features.size(0), -1)
        features = self.fc(features)
        return features


class LSTMDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, encoder_out, targets):
        embeddings = self.embedding(targets)
        h0 = encoder_out.unsqueeze(0)
        c0 = torch.zeros_like(h0)
        outputs, _ = self.lstm(embeddings, (h0, c0))
        logits = self.fc(outputs)
        return logits


class CNNLSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.encoder = CNNEncoder(encoded_dim=hidden_dim)
        self.decoder = LSTMDecoder(vocab_size, embed_dim, hidden_dim)

    def forward(self, images, targets):
        features = self.encoder(images)
        logits = self.decoder(features, targets)
        return logits

    @torch.no_grad()
    def generate(self, image, max_len=100, sos_idx=1, eos_idx=2):
        self.eval()
        features = self.encoder(image)
        h = features.unsqueeze(0)
        c = torch.zeros_like(h)
        token = torch.tensor([[sos_idx]], device=image.device)
        output_tokens = []
        for _ in range(max_len):
            emb = self.decoder.embedding(token)
            out, (h, c) = self.decoder.lstm(emb, (h, c))
            logits = self.decoder.fc(out)
            next_token = logits.argmax(dim=-1)
            if next_token.item() == eos_idx:
                break
            output_tokens.append(next_token.item())
            token = next_token
        return output_tokens

## 3. Training

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-3

loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model = CNNLSTMModel(vocab_size=VOCAB_SIZE).to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total params: {total_params:,} (all trainable)")

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
checkpoint_dir = Path("checkpoints")
checkpoint_dir.mkdir(exist_ok=True)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch_idx, (imgs, seqs) in enumerate(loader):
        imgs = imgs.to(DEVICE)
        seqs = seqs.to(DEVICE)

        input_tokens = seqs[:, :-1]
        target_tokens = seqs[:, 1:]

        optimizer.zero_grad()
        logits = model(imgs, input_tokens)
        loss = criterion(logits.reshape(-1, VOCAB_SIZE), target_tokens.reshape(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if batch_idx % 50 == 0:
            print(f"  Batch {batch_idx}/{len(loader)} | Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}/{EPOCHS} | Avg Loss: {avg_loss:.4f}")

    torch.save({
        "epoch": epoch + 1,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "loss": avg_loss,
    }, checkpoint_dir / f"cnn_epoch_{epoch+1}.pt")

torch.save(model.state_dict(), checkpoint_dir / "cnn_final.pt")
print("Training complete!")

## 4. Save tokenizer

In [ ]:
class CharTokenizer:
    def __init__(self, word_index):
        self.word_index = word_index

with open("latex_tokenizer.pkl", "wb") as f:
    pickle.dump(CharTokenizer(word_index), f)

with open("vocab_size.txt", "w") as f:
    f.write(str(VOCAB_SIZE))

print(f"Saved tokenizer and vocab_size={VOCAB_SIZE}")

## 5. Evaluation on test set

In [ ]:
def normalized_edit_distance(pred, target):
    m, n = len(pred), len(target)
    if m == 0 and n == 0:
        return 0.0
    dp = list(range(n + 1))
    for i in range(1, m + 1):
        prev = dp[0]
        dp[0] = i
        for j in range(1, n + 1):
            temp = dp[j]
            if pred[i - 1] == target[j - 1]:
                dp[j] = prev
            else:
                dp[j] = 1 + min(dp[j], dp[j - 1], prev)
            prev = temp
    return dp[n] / max(m, n)


NUM_EVAL = 50
SEED = 42

test_ds = ds["test"]
random.seed(SEED)
eval_indices = random.sample(range(len(test_ds)), NUM_EVAL)
eval_samples = [test_ds[i] for i in eval_indices]

eval_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

model.eval()
exact_matches = 0
total_edit_dist = 0.0

print(f"Evaluating on {NUM_EVAL} test samples...")
print("-" * 60)

for i, sample in enumerate(eval_samples):
    ground_truth = sample["latex"]
    img = sample["image"].convert("L")
    img_tensor = eval_transform(img).unsqueeze(0).to(DEVICE)  # (1, 1, 128, 128)

    pred_tokens = model.generate(img_tensor)
    prediction = "".join(idx_to_char.get(t, "?") for t in pred_tokens)

    is_exact = prediction == ground_truth
    edit_dist = normalized_edit_distance(prediction, ground_truth)

    if is_exact:
        exact_matches += 1
    total_edit_dist += edit_dist

    status = "EXACT" if is_exact else f"edit_dist={edit_dist:.4f}"
    print(f"  [{i+1}/{NUM_EVAL}] {status}")
    print(f"    GT:   {ground_truth[:80]}")
    print(f"    Pred: {prediction[:80]}")

accuracy = exact_matches / NUM_EVAL
avg_edit_dist = total_edit_dist / NUM_EVAL

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"Model:                    CNN + LSTM")
print(f"Samples:                  {NUM_EVAL}")
print(f"Exact match accuracy:     {accuracy:.2%} ({exact_matches}/{NUM_EVAL})")
print(f"Avg normalized edit dist: {avg_edit_dist:.4f}")